In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

Dataset MNIST dan CIFAR-10

In [16]:
#MNIST
transform_mnist = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

trainset_mnist = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform_mnist)
trainloader_mnist = torch.utils.data.DataLoader(trainset_mnist, batch_size=32, shuffle=True)

testset_mnist = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform_mnist)
testloader_mnist = torch.utils.data.DataLoader(testset_mnist, batch_size=32, shuffle=False)

#CIFAR
transform_cifar = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset_cifar = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_cifar)
trainloader_cifar = torch.utils.data.DataLoader(trainset_cifar, batch_size=32, shuffle=True)

testset_cifar = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_cifar)
testloader_cifar = torch.utils.data.DataLoader(testset_cifar, batch_size=32, shuffle=False)


Files already downloaded and verified
Files already downloaded and verified


CNN Model

In [7]:
class CNN(nn.Module):
    def __init__(self, num_classes=10):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 64 * 7 * 7)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [12]:
class CNN_cifar(nn.Module):
    def __init__(self, num_classes=10):
        super(CNN_cifar, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 64 * 8 * 8)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x


Train Model

In [8]:
def train_model(model, trainloader, num_epochs=5, learning_rate=0.001):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    for epoch in range(num_epochs):
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            if i % 100 == 99:
                print(f'Epoch {epoch + 1}, Batch {i + 1}, Loss: {running_loss / 100:.3f}')
                running_loss = 0.0

Evaluasi model

In [9]:
def evaluate_model(model, testloader):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Akurasi: {100 * correct / total:.2f}%')

MNIST

In [10]:
model_mnist = CNN(num_classes=10)
train_model(model_mnist, trainloader_mnist)
evaluate_model(model_mnist, testloader_mnist)

Epoch 1, Batch 100, Loss: 0.864
Epoch 1, Batch 200, Loss: 0.227
Epoch 1, Batch 300, Loss: 0.142
Epoch 1, Batch 400, Loss: 0.134
Epoch 1, Batch 500, Loss: 0.122
Epoch 1, Batch 600, Loss: 0.115
Epoch 1, Batch 700, Loss: 0.098
Epoch 1, Batch 800, Loss: 0.082
Epoch 1, Batch 900, Loss: 0.086
Epoch 1, Batch 1000, Loss: 0.070
Epoch 1, Batch 1100, Loss: 0.092
Epoch 1, Batch 1200, Loss: 0.059
Epoch 1, Batch 1300, Loss: 0.063
Epoch 1, Batch 1400, Loss: 0.075
Epoch 1, Batch 1500, Loss: 0.071
Epoch 1, Batch 1600, Loss: 0.053
Epoch 1, Batch 1700, Loss: 0.061
Epoch 1, Batch 1800, Loss: 0.066
Epoch 2, Batch 100, Loss: 0.060
Epoch 2, Batch 200, Loss: 0.034
Epoch 2, Batch 300, Loss: 0.046
Epoch 2, Batch 400, Loss: 0.049
Epoch 2, Batch 500, Loss: 0.043
Epoch 2, Batch 600, Loss: 0.043
Epoch 2, Batch 700, Loss: 0.045
Epoch 2, Batch 800, Loss: 0.037
Epoch 2, Batch 900, Loss: 0.044
Epoch 2, Batch 1000, Loss: 0.037
Epoch 2, Batch 1100, Loss: 0.040
Epoch 2, Batch 1200, Loss: 0.042
Epoch 2, Batch 1300, Loss: 0

CIFAR-10

In [13]:
model_cifar = CNN_cifar(num_classes=10)
train_model(model_cifar, trainloader_cifar)
evaluate_model(model_cifar, testloader_cifar)

Epoch 1, Batch 100, Loss: 1.966
Epoch 1, Batch 200, Loss: 1.641
Epoch 1, Batch 300, Loss: 1.526
Epoch 1, Batch 400, Loss: 1.424
Epoch 1, Batch 500, Loss: 1.373
Epoch 1, Batch 600, Loss: 1.341
Epoch 1, Batch 700, Loss: 1.305
Epoch 1, Batch 800, Loss: 1.218
Epoch 1, Batch 900, Loss: 1.235
Epoch 1, Batch 1000, Loss: 1.185
Epoch 1, Batch 1100, Loss: 1.179
Epoch 1, Batch 1200, Loss: 1.121
Epoch 1, Batch 1300, Loss: 1.054
Epoch 1, Batch 1400, Loss: 1.081
Epoch 1, Batch 1500, Loss: 1.077
Epoch 2, Batch 100, Loss: 1.002
Epoch 2, Batch 200, Loss: 1.001
Epoch 2, Batch 300, Loss: 0.938
Epoch 2, Batch 400, Loss: 0.928
Epoch 2, Batch 500, Loss: 0.942
Epoch 2, Batch 600, Loss: 0.902
Epoch 2, Batch 700, Loss: 0.947
Epoch 2, Batch 800, Loss: 0.937
Epoch 2, Batch 900, Loss: 0.933
Epoch 2, Batch 1000, Loss: 0.918
Epoch 2, Batch 1100, Loss: 0.897
Epoch 2, Batch 1200, Loss: 0.878
Epoch 2, Batch 1300, Loss: 0.895
Epoch 2, Batch 1400, Loss: 0.904
Epoch 2, Batch 1500, Loss: 0.869
Epoch 3, Batch 100, Loss: 0.